In [5]:
import numpy as np
import yaml
from astropy.table import Table
from astropy.io import fits
_targetbitdefs = yaml.load("""
#- target pixel mask
targetmask:
    - [QSOLYA,       0, "Lyman alpha QSO"]
    - [QSO,       1, "Tracer QSO"]
    - [LRG,      2, "Luminous Red Galaxy"]
    - [ELG, 3, "Emission Line Galaxy"]
    - [STAR,    4, "Standard Star"]
    - [SKY, 5, "Sky"] 
    - [BGS, 6, "Bright Galaxy"]
""")
import desispec.maskbits as masks

In [6]:
targetmask = masks.BitMask("targetmask", _targetbitdefs)
n_tiles = 10
target_per_tile = 25000
total_n_targets = target_per_tile * n_tiles / 2
random_ID_list =  np.arange(total_n_targets, dtype='int64')
random_ra_list = np.random.random(total_n_targets)
random_dec_list = np.random.random(total_n_targets)
random_type_A_list = np.int_(np.random.random(total_n_targets)*7)
random_type_B_list = np.int_(np.random.random(total_n_targets)*7)
print np.size(np.where(random_type_A_list == random_type_B_list))

17819


# TARGET FORMAT

In [7]:
def write_target_fits(id_list, ra_list, dec_list, type_list, filename):
    t = Table([id_list, ra_list, dec_list, type_list], names=('ID', 'RA', 'DEC', 'TYPE'), meta={'name': 'target table'})
    t.write(filename, format='fits', overwrite=True)

def read_target_fits(filename):
    t = Table.read(filename, format='fits')
    return t
    
for i_tile in range(n_tiles):
    filename = 'tmp/targets_%06d.fits'%(i_tile)
    tile_id = np.random.choice(random_ID_list, size=target_per_tile)
    tile_ra = random_ra_list[tile_id]
    tile_dec = random_dec_list[tile_id]
    tile_type_A = random_type_A_list[tile_id]
    tile_type_B = random_type_B_list[tile_id] 
    tile_type = np.ones(target_per_tile, dtype='int64')
    
    for i in range(target_per_tile):
        tile_type[i]  = targetmask.mask(random_type_A_list[i]) | targetmask.mask(random_type_B_list[i])
    write_target_fits(tile_id, tile_ra, tile_dec, tile_type, filename)
    t = read_target_fits(filename)